# Quick Guide

**Toytree** is a Python tree plotting library designed for use inside 
jupyter notebooks. In fact, this entire tutorial was created using notebooks, and assumes that you are following along in a notebook of your own. To begin, we will import `toytree`, and the plotting library it is built on, `toyplot`, as well as `numpy` for generating some numerical data. 

In [1]:
import toytree       # a tree plotting library
import toyplot       # a general plotting library
import numpy as np   # numerical library

In [2]:
print(toytree.__version__)
print(toyplot.__version__)
print(np.__version__)

0.1.16
0.17.0
1.14.2


### Load and draw your first tree
The main Class object is toytree is a `ToyTree`, which provides plotting functionality in addition to a number of useful functions and attributes for returning values and statistics about trees. As we'll see below, you can generate a ToyTree object in many ways, but generally it is done by reading in a newick formatted string of text. The example below shows the simplest way to load a ToyTree which is to use the `toytree.tree()` convenience function to parse a file, URL, or string.

In [3]:
# load a toytree from a newick string at a URL
tre = toytree.tree("https://eaton-lab.org/data/Cyathophora.tre")

In [4]:
# root and draw the tree (more details on this coming up...)
rtre = tre.root(wildcard='prz')
rtre.draw(width=400, tip_labels_align=True);

38362_rex 39618_rex 35236_rex 35855_rex 40578_rex 30556_thamno 33413_thamno 41954_cyathophylloides 41478_cyathophylloides 30686_cyathophylla 29154_superba 33588_przewalskii 32082_przewalskii

ToyTrees can be flexibly loaded from a range of text formats. Below are two newick strings in different formats. The first has edge lengths and support values, the second has edge-lengths and node-labels. These are two different ways of writing tree data in a serialized format. To parse either format you can tell toytree the format of the newick string following the [tree parsing formats in ete](http://etetoolkit.org/docs/latest/tutorial/tutorial_trees.html#reading-and-writing-newick-trees). The default option, and most common format is 0. If you don't enter a `tree_format` argument the default format will usually parse it just fine. Toytree can also parse extended newick format (nhx) files, which store additional metadata.  

In [5]:
# newick with edge-lengths & support values
newick = "((a:1,b:1)90:3,(c:3,(d:1, e:1)100:2)100:1)100;"
tre0 = toytree.tree(newick, tree_format=0)

# newick with edge-lengths & string node-labels
newick = "((a:1,b:1)A:3,(c:3,(d:1, e:1)B:2)C:1)root;"
tre1 = toytree.tree(newick, tree_format=1)

### Accessing tree data

You can use tab-completion by typing the name of the tree variable (e.g., `rtre` below) followed by a dot and then pressing `<tab>` to see the many attributes of ToyTrees. Below I print a few of them as examples. 

In [6]:
rtre.ntips

13

In [7]:
rtre.nnodes

25

In [8]:
tre.is_rooted(), rtre.is_rooted()

(False, True)

In [9]:
rtre.get_tip_labels()

['38362_rex',
 '39618_rex',
 '35236_rex',
 '35855_rex',
 '40578_rex',
 '30556_thamno',
 '33413_thamno',
 '41954_cyathophylloides',
 '41478_cyathophylloides',
 '30686_cyathophylla',
 '29154_superba',
 '33588_przewalskii',
 '32082_przewalskii']

In [10]:
rtre.get_edge_lengths()

[0.0,
 0.01793711304874569,
 0.01793711304874569,
 0.002976261492013168,
 0.005387231123547946,
 0.002379947556040018,
 0.00941020878048287,
 0.0010337965749144117,
 0.0007833654999050272,
 0.0022299965023919134,
 0.000738900380519168,
 0.00617527349892385,
 0.0025932635060409203,
 0.0024713396985738146,
 0.0063423682644789,
 0.006699449889235297,
 5.282184335845886e-05,
 8.888031167776559e-05,
 0.005653582588384286,
 0.00653218253974622,
 0.003354061346909988,
 0.003399633024335466,
 0.005805246934037405,
 0.0009620811816274587,
 0.001092184346131941]

### Tree Classes

The main Class objects in toytree exist as a nested hierarchy. The core of any tree is the `TreeNode` object, which stores the tree structure in memory and allows fast traversal over nodes of the tree to describe its structure. This object is wrapped inside of `ToyTree` objects, which provide convenient access to TreeNodes while also providing plotting and tree modification functions. And multiple ToyTrees can be grouped together into `MultiTree` objects, which are useful for iterating over multiple trees, or for generating plots that overlay and compare trees. 

The underlying TreeNode object of Toytrees will be familiar to users of the [ete3](http://etetoolkit.org/docs/latest/tutorial/tutorial_trees.html) Python library, since it is simply a stripped-down version/fork of their Tree data structure object. This is useful since ete has great documentation. You can access the TreeNode of any ToyTree using its `.tree` attribute, like below. However, toytree user's are unlikely to need to access TreeNode objects directly, and are instead expected to access it through the ToyTree object.

In [11]:
# a TreeNode object is contained within every ToyTree at .tree
tre.treenode

Tree node 'i23' (-0x7ffff8027fe8ea38)

In [12]:
# a ToyTree object
toytree.tree("((a, b), c);")

In [13]:
# a MultiTree object
toytree.mtree([tre, tre, tre])

### Drawing trees: basics

When you call `.draw()` on a tree it returns two objects, a `Canvas` and a `Cartesian` axes object. These two objects are from `toyplot`, the plotting library toytree is based on. 
The Canvas describes the plot space, and the Cartesian grid arranges our drawing into coordinates on that space. 

As you will see below, I end many toytree drawing commands with a semicolon (;), this simply hides the printed return statement showing that the Canvas and Cartesian objects were returned. The Canvas will automatically render in the cell below the plot even if you do not save the return Canvas as a variable. Below I do not use a semicolon and so the two returned objects are shown as text (e.g., <toyplot.canvas.Canvas...>), and the plot is displayed. 

In [14]:
rtre.draw()

(<toyplot.canvas.Canvas at 0x7fd7ffeb7eb8>,
 <toyplot.coordinates.Cartesian at 0x7fd7ffeb7cf8>)

38362_rex 39618_rex 35236_rex 35855_rex 40578_rex 30556_thamno 33413_thamno 41954_cyathophylloides 41478_cyathophylloides 30686_cyathophylla 29154_superba 33588_przewalskii 32082_przewalskii

In [15]:
# the semicolon hides the returned text of the Canvas and Cartesian objects
rtre.draw();

38362_rex 39618_rex 35236_rex 35855_rex 40578_rex 30556_thamno 33413_thamno 41954_cyathophylloides 41478_cyathophylloides 30686_cyathophylla 29154_superba 33588_przewalskii 32082_przewalskii

In [16]:
# or, we can store them as variables (this allows more editing on them later)
canvas, axes = rtre.draw()

38362_rex 39618_rex 35236_rex 35855_rex 40578_rex 30556_thamno 33413_thamno 41954_cyathophylloides 41478_cyathophylloides 30686_cyathophylla 29154_superba 33588_przewalskii 32082_przewalskii

### Drawing trees: styles
There are innumerous ways in which to style ToyTree drawings. We provide a number of pre-built `tree_styles` (normal, dark, coalescent, multitree), but users can also create their own style dictionaries that can be easily reused. Below are some examples. You can use tab-completion within the draw function to see the docstring for more details on available arguments to toggle, or you can see which styles are available on ToyTrees by accessing their `.style` dictionary. See the [Styling](https://toytree.readthedocs.io/Styling.html) chapter for more details.

In [17]:
# drawing with pre-built tree_styles
rtre.draw(tree_style='normal');
rtre.draw(tree_style='dark');

38362_rex 39618_rex 35236_rex 35855_rex 40578_rex 30556_thamno 33413_thamno 41954_cyathophylloides 41478_cyathophylloides 30686_cyathophylla 29154_superba 33588_przewalskii 32082_przewalskii

38362_rex 39618_rex 35236_rex 35855_rex 40578_rex 30556_thamno 33413_thamno 41954_cyathophylloides 41478_cyathophylloides 30686_cyathophylla 29154_superba 33588_przewalskii 32082_przewalskii

In [18]:
# define a style dictionary
mystyle = {
    "orient": "down",
    "edge_type": 'c',
    "edge_style": {
        "stroke": toytree.colors[2],
        "stroke-width": 3,
    },
    "tip_labels_align": True, 
    "tip_labels_color": toytree.colors[0],
    "tip_labels_style": {
        "font-size": "10px"
    },
    "node_labels": False,
    "node_size": 8,
    "node_color": toytree.colors[2],
}

In [ ]:
# use your custom style dictionary in one or more tree drawings
rtre.draw(height=400, **mystyle);

### Drawing trees: nodes

Plotting node values on a tree is a useful way of representing additional information about trees. Toytree tries to make this process fool-proof, in the sense that the data you plot on nodes will always be the correct data associated with that node. This is done through simple shortcut methods for plotting node features, as well as a convenience function called `.get_node_values()` that draws the values explicitly from the same tree structure that is being plotted (this avoids making a list of values from a tree and then plotting them on that tree only to find that a the order of tips or nodes in the tree has changed.) Finally, toytree also provides interactive features that allow you to explore many features of your data by simply hovering over nodes with your cursor. This is made possible by the HTML+JS framework in which toytrees are displayed in jupyter notebooks, or in web-pages.  

In [ ]:
# hover over nodes to see pop-up elements
rtre.draw(width=500, height=400, node_labels=True, node_size=15, tip_labels_align=True);

In the example above the labels on each node indicate their "idx" value, which is simply a unique identifier given to every node. We could alternatively select one of the features that you could see listed on the node when you hovered over it and toytree will display that value on the node instead. In the example below we plot the node support values. You'll notice that in this context no values were shown for the tip nodes, but instead only for internal nodes. More on this below.

In [ ]:
rtre.draw(node_labels='support');

You can also create plots with the nodes shown, but without node labels. This is often most useful when combined with mapping different colors to nodes to represent different classes of data (see [Color Mapping]()). In the example below we pass a single color and size for all nodes. 

In [ ]:
# You can do the same without printing the 'idx' label on nodes.
rtre.draw(
    node_labels=None,
    node_size=10,
    node_color='grey'
    );

You can draw values on all the nodes, or only on non-tip nodes, or only on internal nodes (not tips or root). Use the `.get_node_values` function of ToyTrees to build a list of values for plotting on the tree. Because the data are extracted from the same tree they will be plotted on the values will always be ordered properly. 

In [ ]:
tre0.get_node_values("support", show_root=1, show_tips=1)

In [ ]:
tre0.get_node_values("support", show_root=1, show_tips=0)

In [ ]:
tre0.get_node_values("support", show_root=0, show_tips=0)

In [ ]:
# show support values 
tre0.draw(
    node_labels=tre0.get_node_values("support", 0, 0),
    node_size=20,
    );

Because `.get_node_values()` returns values in node plot order, it is especially useful for building lists of values for color mapping on nodes. Here we map different colors to nodes depending on whether the support value is 100 or not. 

In [ ]:
# build a color list in node plot order with different values based on support
colors = [
    toytree.colors[0] if i==100 else toytree.colors[1] 
    for i in rtre.get_node_values('support', 1, 1)
]

# You can do the same without printing the 'idx' label on nodes.
rtre.draw(
    node_size=10,
    node_color=colors
    );

### Drawing: saving figures
Toytree drawings can be saved to disk using the `render` functions of toyplot. This is where it is useful to store the Canvas object as a variable when it is returned during a toytree drawing. You can save toyplot figures in a variety of formats, including HTML (which is actually an SVG figures wrapped in HTML with addition javascript to provide interactivity); or SVG, PDF, and PNG. 

In [ ]:
# draw a plot and store the Canvas object to a variable
canvas, axes = rtre.draw(width=400, height=300);

HTML rendering is the default format. This will save the figure as a vector graphic (SVG) wrapped in HTML with additional optional javascript wrapping for interactive features. You can share the file with others and anyone can open it in a browser. You can embed it on your website, or even display it in emails!

In [ ]:
# for sharing through web-links (or even email!) html is great!
toyplot.html.render(canvas, "tree-plot.html")

Optional formats: If you want to do additional styling of your figures in a Illustrator or InkScape then SVG is likely your best option. You can save figures in SVG by simply importing this as an additional option from toyplot. 

In [ ]:
# for creating scientific figures SVG is often the most useful format
import toyplot.svg
toyplot.svg.render(canvas, "tree-plot.svg")

Despite the advantages of working with the SVG or HTML formats (e.g., vector graphics and interactive pop-ups), if you're like me you still sometimes love to have an old-fashioned PDF. Again, you can import this from toyplot. If you installed toytree using conda then this will option will work fine, if you used pip for installation then you may need to install an additional Python package to render PDFs called reportlab. 

In [ ]:
import toyplot.pdf
toyplot.pdf.render(canvas, "tree-plot.pdf")

### Drawing: The Canvas, Axes, and coordinates  
When you call the `toytree.draw()` function it returns two Toyplot objects which are used to display the figure. The first is the Canvas, which is the HTML element that holds the figure, and the second is a Cartesian axes object, which represent the coordinates for the plot. You can catch these objects when they are returned by the `draw()` function to further manipulate the plot. Here we show the axes and add some color. 

In [ ]:
# store the returned Canvas and Axes objects
canvas, axes = rtre.draw(width=350, height=300)

# show the axes coordinates
axes.show = True
axes.x.ticks.show = True
axes.y.ticks.show = True

Or, instead of catching the canvas and axes auto-generated by the `toytree.draw()` function you can instead generate the canvas and axes yourself using Toyplot and pass the axes object as an argument to `draw()` to embed the tree within the axes coordinates. This is a useful way to combine multiple figures on a single canvas, or to annotate axes.

In [ ]:
style = {
    "tip_labels_align": True,
    "tip_labels_style": {
        "font-size": "9px"
    },
}

In [ ]:
# set dimensions of the canvas
canvas = toyplot.Canvas(width=700, height=250)

# dissect canvas into multiple cartesian areas (x1, x2, y1, y2)
ax0 = canvas.cartesian(bounds=('10%', '50%', '10%', '90%'))
ax1 = canvas.cartesian(bounds=('50%', '90%', '10%', '90%'))

# call draw with the 'axes' argument to pass it to a specific cartesian area
rtre.draw(axes=ax0, **style);
rtre.draw(axes=ax1, tip_labels_color='darkred', **style);

# hide the axes (e.g, ticks and splines)
ax0.show=False
ax1.show=False

There's much more that you can do in toytree. Please explore the rest of the documentation and check the Cookbook section for inspiration. 